In [107]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 

from sklearn.model_selection import(
    train_test_split, 
    cross_val_predict,
    KFold,
)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import (
    Dataset, 
    DataLoader,
    TensorDataset,
    random_split,
    SubsetRandomSampler,
    ConcatDataset,
)
from torchvision import transforms, datasets


torch.cuda.is_available()

True

### GPU HELL YEAH!

In [108]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print('Rolling on GPU, babe')
        return device
    print('CPU it is...')
    return torch.device('cpu')

device = get_device()

Rolling on GPU, babe


## Data


In [109]:
# loading data
raw_data = loadarff('mnist_784.arff')
df = pd.DataFrame(raw_data[0])

In [110]:
#validação
portion_to_validate = 0.1
df_validation = df.sample(frac = portion_to_validate)
df = df.drop(df_validation.index)

In [111]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

def train_epoch(net, device, dataloader, loss_fn, optimizer):
    net.train()
    for X, y in dataloader:
        y = y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        output = net(X)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
    return loss

def valid_epoch(net, device, dataloader, loss_fn):
    valid_loss, val_correct = 0.0, 0
    net.eval()
    for X, y in dataloader:
        y = y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)

        output = net(X)
        loss=loss_fn(output, y)
        scores, predictions = torch.max(output.data,1)
        val_correct += (predictions == y).sum().item()

    return loss, val_correct

In [112]:
class MD(Dataset):
 
  def __init__(self, X, y):
    self.X_train = torch.tensor(X.values, dtype=torch.float)
    self.y_train = torch.tensor(y.values.flatten(), dtype=torch.float)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.X_train[idx], self.y_train[idx]

In [114]:
# Separando dataset
X = df.iloc[:, 0:-1]
y = df.iloc[:, [-1]].astype('int')

kf = KFold(n_splits=10)
EPOCHS = 10

# Training the model
fold = 0
for train_index, test_index in kf.split(X, y):  
    train = MD(X.iloc[train_index], y.iloc[train_index])
    test = MD(X.iloc[test_index], y.iloc[test_index])

    train_loader = DataLoader(train, batch_size=128)
    test_loader = DataLoader(test, batch_size=128)

    net = MLP().to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    fold += 1
    print(f'Fold {fold}')

    for epoch in range(EPOCHS):
        train_loss = train_epoch(net, device, train_loader,
                                F.nll_loss, optimizer)

        test_loss, test_correct = valid_epoch(net, device, test_loader,
                                F.nll_loss)

        print(f'Epoch: {epoch+1}/{EPOCHS} Training Loss: {train_loss:.3f} \t Test Accuracy: {test_correct/len(test_loader.sampler)}') 



Fold 1
Epoch: 1/10 Training Loss: 0.169 	 Test Accuracy: 0.9466666666666667
Epoch: 2/10 Training Loss: 0.178 	 Test Accuracy: 0.9623809523809523
Epoch: 3/10 Training Loss: 0.057 	 Test Accuracy: 0.9653968253968254
Epoch: 4/10 Training Loss: 0.047 	 Test Accuracy: 0.9503174603174603
Epoch: 5/10 Training Loss: 0.167 	 Test Accuracy: 0.966031746031746
Epoch: 6/10 Training Loss: 0.037 	 Test Accuracy: 0.9655555555555555
Epoch: 7/10 Training Loss: 0.015 	 Test Accuracy: 0.967936507936508
Epoch: 8/10 Training Loss: 0.008 	 Test Accuracy: 0.9736507936507937
Epoch: 9/10 Training Loss: 0.160 	 Test Accuracy: 0.9588888888888889
Epoch: 10/10 Training Loss: 0.040 	 Test Accuracy: 0.9696825396825397
Fold 2
Epoch: 1/10 Training Loss: 0.276 	 Test Accuracy: 0.9512698412698413
Epoch: 2/10 Training Loss: 0.148 	 Test Accuracy: 0.9488888888888889
Epoch: 3/10 Training Loss: 0.238 	 Test Accuracy: 0.9593650793650793
Epoch: 4/10 Training Loss: 0.150 	 Test Accuracy: 0.9542857142857143
Epoch: 5/10 Training 

In [117]:
# Validations
X_train = df.iloc[:, 0:-1]
y_train = df.iloc[:, [-1]].astype('int')

X_val = df_validation.iloc[:, 0:-1]
y_val = df_validation.iloc[:, [-1]].astype('int')

train = MD(X_train, y_train)
val = MD(X_val, y_val)

train_loader = DataLoader(train, batch_size=128)
val_loader = DataLoader(val, batch_size=128)

net = MLP().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    train_loss = train_epoch(net, device, train_loader,
                            F.nll_loss, optimizer)

    print(f'Epoch: {epoch+1}/{EPOCHS} Training Loss: {train_loss:.3f}')


Epoch: 1/10 Training Loss: 0.221
Epoch: 2/10 Training Loss: 0.030
Epoch: 3/10 Training Loss: 0.010
Epoch: 4/10 Training Loss: 0.006
Epoch: 5/10 Training Loss: 0.070
Epoch: 6/10 Training Loss: 0.061
Epoch: 7/10 Training Loss: 0.000
Epoch: 8/10 Training Loss: 0.656
Epoch: 9/10 Training Loss: 0.000
Epoch: 10/10 Training Loss: 0.031


In [118]:
correct = 0
total = 0

with torch.no_grad():
    for data in val_loader:
        X, y = data
        X, y = X.to(device), y.to(device)
        output = net(X)
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.967
